In [3]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.jobkorea.co.kr/Recruit/Home/_GI_List/'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
    "Referer": "https://www.jobkorea.co.kr/recruit/joblist?menucode=local&localorder=1",
    "X-Requested-With": "XMLHttpRequest",
    "Origin": "https://www.jobkorea.co.kr"
}

payload = {
      "page": 1,
      "order": 2,
      "direct": 0,
      "pagesize": 40,
      "tabindex": 0,
      "onePick": 0,
      "confirm": 0,
      "profile": 0
    }

response = requests.post(url, headers=headers, data=payload)
soup = BeautifulSoup(response.text, "html.parser")
#soup

### 채용리스트에서 페이지별로 채용공고 스크래핑

In [2]:
def get_page_soup(page):
    url = 'https://www.jobkorea.co.kr/Recruit/Home/_GI_List/'
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
    "Referer": "https://www.jobkorea.co.kr/recruit/joblist?menucode=local&localorder=1",
    "X-Requested-With": "XMLHttpRequest",
    "Origin": "https://www.jobkorea.co.kr"
    }

    payload = {
      "page": page,
      "order": 2,
      "direct": 0,
      "pagesize": 40,
      "tabindex": 0,
      "onePick": 0,
      "confirm": 0,
      "profile": 0
    }
    response = requests.post(url, headers=headers, data=payload)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup

In [5]:
get_page_soup(50)


<div class="tplJobListWrap devTplTabBx">
<h2 class="bn_hd_2 blind">채용공고 리스트</h2>
<input id="hdnGICnt" type="hidden" value="182,519"/>
<div class="tplTabBx jobListTab">
<ul class="tplTab clear" id="anchorGICnt_50">
<li class="on" data-tab-index="0"><button onclick="GA_Virtual('홈&gt;채용정보&gt;상세검색_전체', document.URL.replace('jobkorea.co.kr/','jobkorea.co.kr/virtual/') + '&amp;tab=all');" type="button"><span data-text="전체">전체 <em>(182,519건)</em></span></button></li>
<li data-tab-index="1"><button onclick="GA_Virtual('홈&gt;채용정보&gt;상세검색_대기업', document.URL.replace('jobkorea.co.kr/','jobkorea.co.kr/virtual/') + '&amp;tab=major');" type="button"><span data-text="대기업">대기업</span></button></li>
<li data-tab-index="2"><button onclick="GA_Virtual('홈&gt;채용정보&gt;상세검색_중견·중소', document.URL.replace('jobkorea.co.kr/','jobkorea.co.kr/virtual/') + '&amp;tab=SME');" type="button"><span data-text="중견·중소">중견·중소</span></button></li>
<li data-tab-index="3"><button onclick="GA_Virtual('홈&gt;채용정보&gt;상세검색_공기업·공사', d

### 채용리스트에 있는 채용공고 스크래핑

In [4]:
from datetime import datetime

def extract_announcement(soup):
    BASE_URL = 'https://www.jobkorea.co.kr'
    announcement = []
    job_list = soup.select('tbody > tr.devloopArea')
    for job in job_list:
        company_tag = job.select_one('td.tplCo a')
        title_tag = job.select_one('td.tplTit strong a')
        area_tag = job.select_one('td.tplTit p.etc span.cell:nth-of-type(3)')
        career_tag = job.select_one('td.tplTit p.etc span.cell:nth-of-type(1)')
        deadline_tag = job.select_one('td.odd span.date span.tahoma')

        company = company_tag.get_text(strip=True) if company_tag else None
        company_link = company_tag['href'] if company_tag and company_tag.has_attr('href') else '링크없음'

        title = title_tag.get_text(strip=True) if title_tag else None
        title_link = title_tag['href'] if title_tag and title_tag.has_attr('href') else '링크없음'

        area = area_tag.get_text(strip=True) if area_tag else None
        career = career_tag.get_text(strip=True) if career_tag else None
        deadline = deadline_tag.get_text(strip=True) if deadline_tag else None
        if deadline:
            deadline = deadline.replace('~', '')
            year = datetime.today().year
            deadline_date = datetime.strptime(f"{year}/{deadline}", "%Y/%m/%d")
            deadline = deadline_date.strftime("%Y-%m-%d")

        announcement.append({
            'company': company,
            'company_link': BASE_URL + company_link,
            'title': title,
            'title_link': BASE_URL+title_link,
            'area': area,
            'career': career,
            'deadline': deadline,
        })
    return announcement

In [5]:
extract_announcement(soup)

[{'company': '㈜에렉스',
  'company_link': 'https://www.jobkorea.co.kr/Recruit/Co_Read/C/12427684',
  'title': '무인주차관제시스템  주차관리(유도&안내)',
  'title_link': 'https://www.jobkorea.co.kr/Recruit/GI_Read/47327958?rPageCode=SL&logpath=21&sn=6&sc=611',
  'area': '서울 강남구',
  'career': '경력무관',
  'deadline': '2025-08-14'},
 {'company': '제이디자인㈜',
  'company_link': 'https://www.jobkorea.co.kr/Recruit/Co_Read/C/39357969',
  'title': '[인테리어 마케팅] 퍼포먼스 마케팅 경력직 직원 채용',
  'title_link': 'https://www.jobkorea.co.kr/Recruit/GI_Read/47327957?rPageCode=SL&logpath=21&sn=6&sc=611',
  'area': '부산 남구',
  'career': '경력',
  'deadline': '2025-09-13'},
 {'company': '㈜아이디에스앤트러스트',
  'company_link': 'https://www.jobkorea.co.kr/Recruit/Co_Read/C/57398',
  'title': '[대웅그룹/idsTrust] 물류시스템 개발 및 운영담당자 경력직 채용',
  'title_link': 'https://www.jobkorea.co.kr/Recruit/GI_Read/47327956?rPageCode=SL&logpath=21&sn=6&sc=611',
  'area': '서울 강남구',
  'career': '경력무관',
  'deadline': '2025-08-14'},
 {'company': '주식회사 라온제나',
  'company_link': 'h

### 채용정보 스크래핑

In [4]:
url = "https://www.jobkorea.co.kr/Recruit/GI_Read/47325430?rPageCode=SL&logpath=21&sn=6&sc=612"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
#soup

In [8]:
def extract_posted_info(url, headers, deadline):

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    BASE_URL = 'https://www.jobkorea.co.kr'
    def get_text_or_none(tag):
        return tag.get_text(strip=True) if tag else None

    def get_all_texts(tags):
        return [t.get_text(strip=True) for t in tags] if tags else None

    def get_next_dd_by_dt(text):
        dt = soup.select_one(f'dt:-soup-contains("{text}")')
        return dt.find_next_sibling('dd') if dt else None

    def get_homepage_url():
        dt = soup.select_one('dt:-soup-contains("홈페이지")')
        if dt:
            dd = dt.find_next_sibling('dd')
            if dd:
                a = dd.select_one('a[href]')
                if a:
                    return a['href']
        return None

    result = {}

    # 채용공고 제목 → title
    title_tag = soup.select_one('article.artReadJobSum h3.hd_3')
    result['title'] = title_tag.get_text(strip=True) if title_tag else None

    # 회사이름
    company_name = soup.select_one('span.coName')
    result['company_name'] = get_text_or_none(company_name)

    # 경력 → career
    career = get_next_dd_by_dt('경력')
    result['career'] = get_text_or_none(career)

    # 학력 → education
    education = get_next_dd_by_dt('학력')
    result['education'] = get_text_or_none(education)

    # 스킬 → skill
    skill = get_next_dd_by_dt('스킬')
    if skill:
        skills = [s.strip() for s in skill.get_text(strip=True).split(',')]
    else:
        skills = None
    result['skill'] = skills

    # 우대 → preferred
    preference_spans = soup.select('dl#dlPref dd span.pref')
    result['preferred'] = get_all_texts(preference_spans)

    # 고용형태 → contract_type
    employment_list = []
    employment_lis = soup.select('dt:-soup-contains("고용형태") + dd li')
    for li in employment_lis:
        employment_list.append(li.get_text(strip=True))
    result['contract_type'] = employment_list if employment_list else None

    # 급여 → salary
    salary = get_next_dd_by_dt('급여')
    result['salary'] = get_text_or_none(salary)

    # 지역 → location
    location = soup.select_one('dt:-soup-contains("지역") + dd a')
    result['location'] = get_text_or_none(location)

    # 시간 → working_time
    working_time = get_next_dd_by_dt('시간')
    result['working_time'] = get_text_or_none(working_time)

    # 직급 → position_level
    position = get_next_dd_by_dt('직급')
    result['position_level'] = get_text_or_none(position)

    # 직책 → position_role
    role = get_next_dd_by_dt('직책')
    result['position_role'] = get_text_or_none(role)

    # 상세요강 content
    iframe_tag = soup.select_one('article.artReadDetail iframe')
    content_url = BASE_URL + iframe_tag['src'] if iframe_tag and iframe_tag.has_attr('src') else None

    result['content'] = extract_posted_content_info(content_url, headers)

    # url
    result['url'] = url

    # 마감일 → due_date
    result['due_date'] = deadline
    # 기업정보 추가
    result['industry'] = get_text_or_none(get_next_dd_by_dt('산업(업종)'))
    result['employees'] = get_text_or_none(get_next_dd_by_dt('사원수'))
    result['founded'] = get_text_or_none(get_next_dd_by_dt('설립년도'))
    result['company_type'] = get_text_or_none(get_next_dd_by_dt('기업형태'))
    result['homepage'] = get_homepage_url()

    return result

In [6]:
# 본문내용
def extract_posted_content_info(url, headers):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    body_text = soup.body.get_text(separator='\n', strip=True)
    return body_text

In [9]:
extract_posted_info(url, headers, None)

{'title': '코웨이㈜기업인증 완료관심기업관심기업 추가하고채용소식 받기닫기[코웨이]7월 계약직 채용(수출입/연구/데이터/생산/실험 업무 지원)',
 'company_name': '코웨이㈜',
 'career': '경력',
 'education': '학력무관',
 'skill': None,
 'preferred': None,
 'contract_type': ['계약직근무기간 12개월'],
 'salary': '회사내규에 따름 \r\n - 면접 후 결정',
 'location': '서울시 구로구',
 'working_time': '주5일\r\n                                    (월~금)09:00~18:00',
 'position_level': None,
 'position_role': None,
 'content': '',
 'url': 'https://www.jobkorea.co.kr/Recruit/GI_Read/47325430?rPageCode=SL&logpath=21&sn=6&sc=612',
 'due_date': None,
 'industry': '그 외 기타 달리 분류되지 않은 개인 서비스업',
 'employees': '6,233명',
 'founded': '1989년 (37년차)',
 'company_type': '대기업\r\n                                    (코스피)',
 'homepage': 'https://company.coway.com'}

### 회사정보 스크래핑

In [10]:
url = "https://www.jobkorea.co.kr/Recruit/Co_Read/C/17021185"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")
#soup

In [31]:
def extract_posted_company_info(url, headers):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    company_name_div = soup.select_one('.company-header-branding-body .name')
    company_name = company_name_div.get_text(strip=True) if company_name_div else None

    def get_value(label):
        ths = soup.find_all('th', class_='field-label')
        for th in ths:
            if th.get_text(strip=True) == label:
                td = th.find_next_sibling('td')
                return td.get_text(strip=True) if td else None
        return None

    return {
        'company_name': company_name,
        'industry': get_value('산업'),
        'company_type': get_value('기업구분'),
        'capital': get_value('자본금'),
        'ceo': get_value('대표자'),
        'insurance': get_value('4대보험'),
        'address': get_value('주소'),
        'employees': get_value('사원수'),
        'founded_date': get_value('설립일'),
        'sales': get_value('매출액'),
        'business_desc': get_value('주요사업'),
        'homepage': get_value('홈페이지'),
    }

In [12]:
extract_posted_company_info(url, headers)

{'industry': '컴퓨터 프로그래밍 서비스업',
 'company_type': '중소기업',
 'capital': '1,000만원(2017.12.31)',
 'ceo': '안흥섭',
 'insurance': '국민연금, 건강보험, 고용보험, 산재보험',
 'address': '서울 강서구 화곡로 429 (가양동, 토피아빌딩) 402-11',
 'employees': '2명(2017.12.31)',
 'founded_date': '2014.07.01(12년차)',
 'sales': '1억 8,899만원(2017.12.31)',
 'business_desc': '컴퓨터프로그래밍,컴퓨터시스템 통합자문,구축,정보기술,컴퓨터운영관련서비스/컴퓨터,주변장치,소프트웨어 소매',
 'homepage': 'http://cafe.naver.com/ianit'}

### Mysql database에 넣기위한 전처리

In [18]:
import json

def preprocess_data_for_mysql(data):
    processed_data = {}
    for key, value in data.items():
        if value is None:
            processed_data[key] = None
        elif isinstance(value, list):
            # 리스트는 JSON 문자열로 변환 (권장)
            processed_data[key] = json.dumps(value, ensure_ascii=False)
        elif isinstance(value, str):
            processed_data[key] = value.strip()
        else:
            processed_data[key] = str(value).strip()
    return processed_data


In [16]:
posted_url = "https://www.jobkorea.co.kr/Recruit/GI_Read/47325430?rPageCode=SL&logpath=21&sn=6&sc=612"
company_url = "https://www.jobkorea.co.kr/Recruit/Co_Read/C/17021185"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

In [22]:
posted = preprocess_data_for_mysql(extract_posted_info(posted_url, headers, None))
#company = preprocess_data_for_mysql(extract_posted_company_info(company_url, headers))

In [23]:
posted

{'title': '코웨이㈜기업인증 완료관심기업관심기업 추가하고채용소식 받기닫기[코웨이]7월 계약직 채용(수출입/연구/데이터/생산/실험 업무 지원)',
 'company_name': '코웨이㈜',
 'career': '경력',
 'education': '학력무관',
 'skill': None,
 'preferred': None,
 'contract_type': '["계약직근무기간 12개월"]',
 'salary': '회사내규에 따름 \r\n - 면접 후 결정',
 'location': '서울시 구로구',
 'working_time': '주5일\r\n                                    (월~금)09:00~18:00',
 'position_level': None,
 'position_role': None,
 'content': '',
 'url': 'https://www.jobkorea.co.kr/Recruit/GI_Read/47325430?rPageCode=SL&logpath=21&sn=6&sc=612',
 'due_date': None,
 'industry': '그 외 기타 달리 분류되지 않은 개인 서비스업',
 'employees': '6,233명',
 'founded': '1989년 (37년차)',
 'company_type': '대기업\r\n                                    (코스피)',
 'homepage': 'https://company.coway.com'}

In [33]:
company

{'company_name': '㈜이안아이티컨설팅',
 'industry': '컴퓨터 프로그래밍 서비스업',
 'company_type': '중소기업',
 'capital': '1,000만원(2017.12.31)',
 'ceo': '안흥섭',
 'insurance': '국민연금, 건강보험, 고용보험, 산재보험',
 'address': '서울 강서구 화곡로 429 (가양동, 토피아빌딩) 402-11',
 'employees': '2명(2017.12.31)',
 'founded_date': '2014.07.01(12년차)',
 'sales': '1억 8,899만원(2017.12.31)',
 'business_desc': '컴퓨터프로그래밍,컴퓨터시스템 통합자문,구축,정보기술,컴퓨터운영관련서비스/컴퓨터,주변장치,소프트웨어 소매',
 'homepage': 'http://cafe.naver.com/ianit'}